# Negentropy Transformer with Seal Protocol

Example usage of the NegentropyTransformerBlock with integrated seal protocol controls.

In [ ]:
import torch
import json
from negentropy_transformer_block import NegentropyTransformerBlock
from audit_log import AuditLog

In [ ]:
# Initialize the transformer block
model = NegentropyTransformerBlock(
    d_model=512,
    nhead=8,
    dim_feedforward=2048,
    dropout=0.1,
    qrh_embed_dim=64,
    alpha=1.0,
    use_learned_rotation=True,
    enable_gate=True
)

# Initialize audit log
audit = AuditLog()

In [ ]:
# Create sample input
batch_size = 2
seq_len = 10
d_model = 512

sample_input = torch.randn(batch_size, seq_len, d_model)
print(f"Input shape: {sample_input.shape}")

In [ ]:
# Forward pass with seal protocol
output, seal = model(sample_input)

print(f"Output shape: {output.shape}")
print(f"Seal generated: {type(seal)}")

In [ ]:
# Log the seal to audit trail
audit.log_entry(seal)

# Calculate latency from seal data (if available in seal)
latency_ms = 50.0  # This would be calculated during forward pass

print("🔐 Output Seal:")
print(json.dumps(seal, indent=2, ensure_ascii=False))

print(f"\n✅ Audit Chain Valid: {audit.validate_chain()}")
print(f"⏱️  Latency: {latency_ms:.2f}ms")
print(f"🎯 RG: {seal['RG']} (Target: 0.347)")
print(f"🌀 Dyad: {seal['active_dyad']}")
print(f"🛡️  Seal: {seal['continuity_seal']}")

In [ ]:
# Check for violations in audit log
violations = audit.count_violations()
print("📊 Violation Summary:")
for violation_type, count in violations.items():
    print(f"  {violation_type}: {count}")

In [ ]:
# Test multiple forward passes to see seal consistency
print("🔄 Testing multiple forward passes:")
for i in range(3):
    output, seal = model(sample_input)
    audit.log_entry(seal)
    
    status = "✅" if not seal.get('latency_sigill', False) else "⚠️"
    print(f"Pass {i+1}: {status} RG={seal['RG']:.3f}, Seal={seal['continuity_seal']}")

print(f"\nFinal audit chain valid: {audit.validate_chain()}")